In [288]:
import pandas as pd
import json
from bs4 import BeautifulSoup,NavigableString, Tag

In [327]:
# import csv file to pandas dataframe
df = pd.read_csv('allData.csv')
df.drop(columns=df.columns[0], axis=1, inplace=True)

In [531]:
df.loc[df['name'] == "Bipolar disorder"]

,name,link,description,header
33,Bipolar disorder,https://www.nhsinform.scot/illnesses-and-condi...,"<div class=""js-guide cf guide"">\n<div class=""c...","[{'title': 'Introduction', 'data': ['<div clas..."


In [248]:
# Remove '[]' symbol in description and get appropriate data
# Do not run again
formatted_description = []
for index, row in df.iterrows():
   fData = row['description'][1:-1]
   soup = BeautifulSoup(fData, 'html.parser')
   mainData = soup.find('div', 'js-guide cf guide')
   if mainData==None:
      editors = soup.find_all('div','editor')
      mainData = ''
      for editor in editors:
         mainData += str(editor)
      mainData = '>div class="js-guide cf guide">' + mainData + '</div>'
   else:
      mainData = str(mainData)
   formatted_description.append(mainData)
df['description'] = pd.Series(formatted_description)

In [320]:
# All headers and their content have been added to the dataframe
# No need to run again
header_words = []
for i,d in df.iterrows():
   soup = BeautifulSoup(d['description'], 'html.parser')
   headers = soup.find_all('h2')
   header_and_data = []
   for header in headers:
      item = {'title':header.text, 'data':[]}
      for sib in header.find_next_siblings():
         if(sib.name=="h2"):
            break
         else:
            item['data'].append(str(sib))
      header_and_data.append(item)
   header_words.append(header_and_data)
header_words = pd.Series(header_words)
df['header'] = header_words

In [347]:
print(any(word in 'This is a test' for word in ['isa']))

False


In [526]:
def determine_typeOfTitle(title):
   if(any(word in title for word in ['symptom','happen','sign','indicator','feature','trait','clue','problem','stroke','pain','feel'])):
      return 'symptom'
   if(any(word in title for word in ['about','defini','descrip','explana','explain','information','introduct','how common','how long','how many','how much'])):
      return 'about'
   elif(any(word in title for word in ['cause','trigger','origin','source','spread','basis','reason','factors''infected'])):
      return 'cause'
   if(any(word in title for word in ['bite','cyst','how do i get'])):
      return 'cause'
   if(any(word in title for word in ['diagnos','Immunophenotyping','pcr','scan','x-ray','test','identify','identifi','detect','discover','screen','support'])):
      return 'diagnosis'
   if(any(word in title for word in ['prevent','rule','precaution','control','mitigation','diet','exercise','tips'])):
      return 'prevention'
   if(any(word in title for word in ['tablet','induction','imatinib','treat','therap','therapeutic','inject','medicat','medicine','consolidat','mainten','biopsy','painkill','surger','etymology','scopy','ology','biotics','puncture','transplant'])):
      return 'treatment'
   if(any(word in title for word in ['who','risk','blood pressure','smoking','history','family','weight','famili','men','women','old','young','depress','genetic','radiation','environment','benzene','alcohol','age'])):
      return 'risks'
   if(any(word in title for word in ['complicat','bleed','infertil','effect','peritonitis','abscess','infect'])):
      return 'complications'
   if(any(word in title for word in ['vaccin'])):
      return 'vaccination'
   if(any(word in title for word in ['medical advice','advice','hospital','getting help','living with'])):
      return 'medical advice'
   if(any(word in title for word in ['emergency'])):
      return 'emergency'
   if(any(word in title for word in ['type','class','category','stage'])):
      return 'types'
   if(any(word in title for word in ['myth'])):
      return 'myths'
   else:
      return 'other'

In [527]:
all_header_types = []
data = []
for i,d in df.iterrows():
   headers = eval(d['header'])
   item = []
   for header in headers:
      title=header['title']
      type = determine_typeOfTitle(title.lower())
      # print(type)
      item.append({'title':title, 'type':type, 'data':header['data'],'disease':d['name']})
      all_header_types.append(type)
   data.extend(item)
all_header_types = pd.Series(all_header_types)
all_header_types.value_counts()

other             1540
treatment         1009
symptom            600
cause              528
diagnosis          507
risks              446
about              339
complications      234
prevention         225
medical advice     214
types               53
vaccination         20
emergency            1
myths                1
dtype: int64

In [ ]:
data = sorted(list(filter(lambda x: len(x['title']) and x['type'],data)), key=lambda x: x['title'])
for d in data:
   if(d['disease']=='Bipolar disorder' and d['type']=='other'):
      print(d['title']+'\n'+d['type']+"-"+d['disease']+'\n'+str(d['data']))  
# result = list(map(lambda x: x['type']+" : "+x['title']+" : "+x['disease'],data))
# sorted(result)[1690:]

In [277]:
value_counts = all_headers.value_counts().to_dict()
value_counts_words = all_words_in_headers.value_counts().to_dict()
value_counts_words
value_counts

{'When to seek medical advice': 72,
 'Surgery': 50,
 'Medication': 42,
 'Introduction': 38,
 'Signs and symptoms': 36,
 'When to see your GP': 33,
 'Causes': 31,
 'Further tests': 31,
 'Chemotherapy': 29,
 'Treatment': 28,
 'Radiotherapy': 27,
 'Who is affected?': 23,
 'Outlook': 23,
 'Complications': 22,
 'Who is affected': 20,
 'Diet': 18,
 'Exercise': 17,
 'Your treatment plan': 17,
 'Symptoms': 16,
 'Blood tests': 14,
 'Seeing your GP': 14,
 'Diagnosis': 14,
 'Other symptoms': 14,
 'Further testing': 14,
 'Biopsy': 14,
 'Your feelings': 14,
 'Referral to a specialist': 14,
 'Follow-up': 14,
 'When to get professional advice': 14,
 'Genetics': 14,
 'Increased risk': 13,
 'Other causes': 13,
 'Antibiotics': 13,
 "Who's affected?": 12,
 'Lifestyle changes': 12,
 'Information about you': 12,
 'Smoking': 12,
 'Seeking medical advice': 11,
 'Pregnancy': 11,
 'Self-help': 11,
 'Family history': 9,
 'Why it happens': 9,
 'Talk to others': 9,
 'When to get medical advice': 9,
 'Alcohol': 9,

In [ ]:
soup = BeautifulSoup(fData[1:-1], 'html.parser')
soup.find('main').findChildren(recursive=False)[1]

In [41]:
about = soup.select("div[id^=about]")
symptoms = soup.select("div[id^=symptoms]")
causes = soup.select("div[id^=causes]")
diagon = soup.select("div[id^=diagnos]")
treat = soup.select("div[id^=treating]")
prevent = soup.select("div[id^=preventing]")

In [ ]:
about

In [127]:
import re
# Iterrate over pandas dataframe
parsed = {'about': [], 'symptoms': [], 'causes': [], 'diagnosis': [], 'treating': [], 'preventing': [],'complications': []}
for index, row in df.iterrows():
   fData = row['description']
   soup = BeautifulSoup(fData[1:-1], 'html.parser')
   def abtCond(x):
      return 'about' in x.lower() or 'introduction' in x.lower()
   about = soup.find("div",id=lambda x: x and abtCond(x))
   # introduction = soup.find("div",id=lambda x: x and 'introduction' in x.lower())
   symptoms = soup.find("div",id=lambda x: x and 'symptoms' in x.lower())
   causes = soup.find("div",id=lambda x: x and 'causes' in x.lower())
   diagon = soup.find("div",id=lambda x: x and 'diagnos' in x.lower())
   treat = soup.find("div",id=lambda x: x and 'treating' in x.lower())
   prevent = soup.find("div",id=lambda x: x and 'preventing' in x.lower())
   complications = soup.find("div",id=lambda x: x and 'complications' in x.lower())
   parsed['symptoms'].append(symptoms)
   parsed['causes'].append(causes)
   parsed['diagnosis'].append(diagon)
   parsed['treating'].append(treat)
   parsed['preventing'].append(prevent)
   parsed['complications'].append(complications)
   # if(index == 4):
   #    print(about)
   #    print(introduction)

In [128]:
# parsed into dataFrame
parsed_df = pd.DataFrame(parsed)
parsed_df.describe()

,about,symptoms,causes,diagnosis,treating,preventing,complications
count,280,143,174,149,191,52,80
unique,280,143,174,149,191,52,80
top,"[\n, [About abdominal aortic aneurysms], \n, [...","[\n, [Symptoms of an abdominal aortic aneurysm...","[\n, [Causes of an abdominal aortic aneurysm],...","[\n, [Diagnosing an abdominal aortic aneurysm]...","[\n, [Treating an abdominal aortic aneurysm], ...","[\n, [Preventing an abdominal aortic aneurysm]...","[\n, [Complications of acne], \n, [\n, [Acne s..."
freq,1,1,1,1,1,1,1


In [129]:
arr = parsed_df['diagnosis'].notnull()
arr.describe()

count       322
unique        2
top       False
freq        173
Name: diagnosis, dtype: object

In [130]:
175-322

-147

In [ ]:
noDiagnosis = parsed_df[parsed_df['diagnosis'].isnull()]
noDiagnosis.head()

In [ ]:
noAbout = parsed_df[parsed_df['about'].isnull()]
noAbout.head()

In [ ]:
parsed_df.iloc[43]

In [134]:
df.iloc[43]

name                                     Brain tumours: Children
link           https://www.nhsinform.scot/illnesses-and-condi...
description    [<main class="site-main" id="maincontent">\n  ...
Name: 43, dtype: object

In [158]:
for index, row in df.iterrows():
   fData = row['description']
   if(fData.startsWith('[')):
      df['description'][index] = fData[1:-1]

TypeError: 'NoneType' object is not callable

In [165]:
df.describe()

,name,link,description
count,322,322,322
unique,322,322,317
top,Abdominal aortic aneurysm,https://www.nhsinform.scot/illnesses-and-condi...,[]
freq,1,1,6


In [224]:
df.tail()

,name,link,description
317,Warts and verrucas,https://www.nhsinform.scot/illnesses-and-condi...,"[\n, [\n, [\n Warts and verrucas\n ..."
318,Whooping cough,https://www.nhsinform.scot/illnesses-and-condi...,"[\n, [\n, [\n Whooping cough\n ..."
319,Wilms’ tumour,https://www.nhsinform.scot/illnesses-and-condi...,"[\n, [\n, [\n Wilms’ tumour\n ..."
320,Womb (uterus) cancer,https://www.nhsinform.scot/illnesses-and-condi...,"[\n, [\n, [\n Womb (uterus) cancer\..."
321,Yellow fever,https://www.nhsinform.scot/illnesses-and-condi...,"<div class=""editor"">\n<p>Yellow fever is a ser..."


In [ ]:
df['description'][43]

In [326]:
df.to_csv('allData.csv')

In [537]:
json.dump(data, open('data.json', 'w'))